In [1]:
# init
from os import path
import numpy as np
from pynput import keyboard
from pedalboard import Pedalboard, Chorus, Reverb, LowpassFilter, load_plugin
from pedalboard.io import AudioFile
import music21
from keyboard_layout import KeyboardLayout
from tone import Tone
from audio_player import AudioPlayer
from time_going import Timeline
from tone import ADSRTone
from midiserial import MidiSerial

smplRate = 44100

keyboard_layout = KeyboardLayout()
keyboard_layout.load('complete left to right')


complete left to right loaded as keyboard layout successfully.


In [2]:
# def scaling_1(point, base):
#     return (250-100*base) * np.power(base,point/12)

# def scaling_1p(point):
#     return 100 * np.power(2,point/31)

# def scaling_2(point):
#     scale = [2,2,1,3,1,2,1]
#     result = 1
#     for i in range(point):
#         result += scale[i%len(scale)]
#     return result

# def final_scaling(x):
#     return scaling_1(scaling_2(x),2)



In [3]:
timeline = Timeline(IS_LOOPED = False)

In [4]:
# effect = load_plugin("/Library/Audio/Plug-Ins/VST3/ValhallaDelay.vst3")
# print(effect.parameters.keys())


In [5]:
# print(effect.parameters)

In [6]:
# effect.bypass = True
# board = Pedalboard([Chorus(), Reverb(room_size=0.25), LowpassFilter(300)])
# board = Pedalboard([effect])
board = Pedalboard([ Reverb(room_size=0.25)])
audioplayer = AudioPlayer(timeline,board)
audioplayer.start()

In [7]:
import time

e = 2
def harmonics_maker():
    e = 2
    rng = np.random.default_rng()
    modifier = rng.random(18)>0.5
    # harmonics = [(np.power(e,np.log2(i)),np.power(e,np.log2(i))) for i in range(2,20)]
    harmonics = np.array([(np.power(e,np.log2(i)),np.power(e,np.log2(i))) for i in range(2,20)])
    harmonics[:,0] = harmonics[:,0]+modifier*10000
    harmonics[:,1] = harmonics[:,1]+modifier*10000
    return harmonics


def scaling_1k(point):
    return 27.5 * np.power(e,point/12)
def scaling_2k(point):
    scale = [2,2,1,2,2,2,1]
    result = 1
    for i in range(point):
        result += scale[i%len(scale)]
    return result
def scaling_k(point):
    return scaling_1k(scaling_2k(point))
# tonemaker_1 = Tone(harmonics = harmonics, fade_duration = 0.06)


# keyboard_layout.start(scaling_1k, harmonics, timeline)
midsr = MidiSerial('/dev/cu.usbserial-00000000')
midsr.start(scaling_1k, harmonics_maker, timeline)


# while True:
#     e = 2
#     rng = np.random.default_rng()
#     modifier = rng.random(18)>0.5
#     # harmonics = [(np.power(e,np.log2(i)),np.power(e,np.log2(i))) for i in range(2,20)]
#     harmonics = np.array([(np.power(e,np.log2(i)),np.power(e,np.log2(i))) for i in range(2,20)])
#     harmonics[:,0] = harmonics[:,0]+modifier*10000
#     harmonics[:,1] = harmonics[:,1]+modifier*10000
#     keyboard_layout.harmonics = harmonics
#     # print(keyboard_layout.harmonics)
#     time.sleep(1)

start:   53  vel: 0.029565439418349264
stop:   53  vel: 64
start:   56  vel: 0.038278489342946115
start:   57  vel: 0.0827484649992508
stop:   56  vel: 64
start:   52  vel: 0.09817591045711065
stop:   57  vel: 64
start:   50  vel: 0.05512102465384241
stop:   52  vel: 64
start:   47  vel: 0.06485163489096522
stop:   50  vel: 64
start:   52  vel: 0.06253283859798868
stop:   47  vel: 64
start:   50  vel: 0.021611196955026633
stop:   52  vel: 64
start:   53  vel: 0.18235854329525236
start:   41  vel: 0.29008876565862923
stop:   50  vel: 64
start:   42  vel: 0.04620596514536203
start:   54  vel: 0.01934575011276403
stop:   53  vel: 64
stop:   41  vel: 64
start:   43  vel: 0.3304256123997345
stop:   54  vel: 64
start:   55  vel: 0.16211302338655498
stop:   42  vel: 64
start:   56  vel: 0.02158993328392195
stop:   55  vel: 64
start:   55  vel: 0.022841268192622735
stop:   56  vel: 64
start:   56  vel: 0.030408268359971815
stop:   55  vel: 64
start:   55  vel: 0.05515787718345466
stop:   56  v

KeyboardInterrupt: 

In [ ]:
# closing audio in case of emergency
audioplayer.stop()

In [ ]:
timeline.clear()

In [ ]:
timeline.save()

In [ ]:
timeline.load()